In [1]:
from pyspark.sql import SparkSession
import os
os.environ['GROUP_ID'] = 'g_teg_ams_exp_ams_exp_report'
os.environ['GAIA_ID'] = '3122'
session = SparkSession.builder.config('spark.driver.memory', '8g').config('spark.executor.cores', 8).config('spark.executor.memory', '8g').getOrCreate()

Driver Log: 686191d9-55f7-4af5-bf8a-d1a58ce1dce6.log
Tracking URL: http://application.tdw.oa.com:8080/proxy/application_1651128848533_18588754


In [2]:
import numpy as np
import os
import pandas as pd
import math
import scipy.stats as stats
from matplotlib import pyplot as plt
expGroupId=55072
expId1=227518
expId2=227520
expId3=227522
expId4=227524
triggerValue=16
triggerId=111017984+triggerValue
beginDate='20220819'
endDate='20220825'
dateRange=[d.strftime('p_%Y%m%d') for d in pd.date_range(beginDate,endDate)]
print(dateRange)

['p_20220819', 'p_20220820', 'p_20220821', 'p_20220822', 'p_20220823', 'p_20220824', 'p_20220825']


In [3]:
from pytoolkit import TDWSQLProvider
provider = TDWSQLProvider(session, group='tl', db='teg_ams_exp_report_app')
provider.table('t_exp_trigger_bucket_d', priParts=dateRange).createOrReplaceTempView('t_exp_trigger_bucket_d')

In [4]:
sqlString="""select bucket_id,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),gmv/topo_width*10000,0))/1000000 as base_gmv_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),gmv/topo_width*10000,0))/1000000 as base_gmv_2,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),gmv/topo_width*10000,0))/1000000 as test_gmv_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),gmv/topo_width*10000,0))/1000000 as test_gmv_2,
       sum(if(exp_id={},gmv/topo_width*10000,0))/1000000 as base_gmv,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_2,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),real_cost_micro_with_contract/topo_width*10000,0))/1000000 as test_cost_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}'),real_cost_micro_with_contract/topo_width*10000,0))/1000000 as test_cost_2,
       sum(if(exp_id={},real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and ocpx_type<3,gmv/topo_width*10000,0))/1000000 as base_gmv_normal_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and ocpx_type<3,gmv/topo_width*10000,0))/1000000 as base_gmv_normal_2,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and ocpx_type<3,gmv/topo_width*10000,0))/1000000 as test_gmv_normal_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and ocpx_type<3,gmv/topo_width*10000,0))/1000000 as test_gmv_normal_2,
       sum(if(exp_id={} and ocpx_type<3,gmv/topo_width*10000,0))/1000000 as base_gmv_normal
from t_exp_trigger_bucket_d
where partition_time between {} and {}
       and process_time between {} and {}
       and exp_id in ({},{},{},{})
       and abnormal_user_type in (0)
group by bucket_id""".format(
    expId1,triggerId,
    expId2,triggerId,
    expId3,triggerId,
    expId4,triggerId,
    expId1,
    expId1,triggerId,
    expId2,triggerId,
    expId3,triggerId,
    expId4,triggerId,
    expId1,
    expId1,triggerId,
    expId2,triggerId,
    expId3,triggerId,
    expId4,triggerId,
    expId1,
    beginDate,endDate,
    beginDate,endDate,
    expId1,expId2,expId3,expId4
)
print(sqlString)

select bucket_id,
       sum(if(exp_id=227518 and array_contains(split(trigger_ids,','),'111018000'),gmv/topo_width*10000,0))/1000000 as base_gmv_1,
       sum(if(exp_id=227520 and array_contains(split(trigger_ids,','),'111018000'),gmv/topo_width*10000,0))/1000000 as base_gmv_2,
       sum(if(exp_id=227522 and array_contains(split(trigger_ids,','),'111018000'),gmv/topo_width*10000,0))/1000000 as test_gmv_1,
       sum(if(exp_id=227524 and array_contains(split(trigger_ids,','),'111018000'),gmv/topo_width*10000,0))/1000000 as test_gmv_2,
       sum(if(exp_id=227518,gmv/topo_width*10000,0))/1000000 as base_gmv,
       sum(if(exp_id=227518 and array_contains(split(trigger_ids,','),'111018000'),real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_1,
       sum(if(exp_id=227520 and array_contains(split(trigger_ids,','),'111018000'),real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_2,
       sum(if(exp_id=227522 and array_contains(split(trigger_ids,','

In [5]:
df=session.sql(sqlString).toPandas()

In [9]:
df

,bucket_id,base_gmv_1,base_gmv_2,test_gmv_1,test_gmv_2,base_gmv,base_cost_1,base_cost_2,test_cost_1,test_cost_2,base_cost,base_gmv_normal_1,base_gmv_normal_2,test_gmv_normal_1,test_gmv_normal_2,base_gmv_normal
0,19,2.699920e+06,3.149606e+06,2.862527e+06,2.854796e+06,5.511973e+07,3.683279e+06,3.672814e+06,3.723195e+06,3.716295e+06,7.151254e+07,2.460363e+06,2.559001e+06,2.579327e+06,2.441152e+06,4.934740e+07
1,0,2.731847e+06,2.805817e+06,2.735690e+06,2.753605e+06,5.489106e+07,3.665204e+06,3.596485e+06,3.713113e+06,3.681172e+06,7.131591e+07,2.441178e+06,2.363257e+06,2.472878e+06,2.440507e+06,4.906279e+07
2,7,2.778007e+06,2.695489e+06,2.740651e+06,3.231631e+06,5.485345e+07,3.670286e+06,3.604112e+06,3.705430e+06,3.725360e+06,7.132747e+07,2.518158e+06,2.302115e+06,2.527324e+06,2.721565e+06,4.912023e+07
3,6,2.704792e+06,2.843111e+06,2.762821e+06,2.929349e+06,5.441919e+07,3.660904e+06,3.587918e+06,3.717831e+06,3.727791e+06,7.137111e+07,2.433533e+06,2.401377e+06,2.492530e+06,2.608905e+06,4.869628e+07
4,9,2.672716e+06,2.917627e+06,2.785915e+06,3.019818e+06,5.464543e+07,3.674566e+06,3.633925e+06,3.726590e+06,3.712681e+06,7.139187e+07,2.425927e+06,2.537116e+06,2.527474e+06,2.650616e+06,4.885776e+07
5,17,2.675148e+06,2.896374e+06,2.826666e+06,3.002610e+06,5.529562e+07,3.664985e+06,3.626730e+06,3.709193e+06,3.725795e+06,7.138505e+07,2.416358e+06,2.377102e+06,2.581065e+06,2.506993e+06,4.937010e+07
6,5,2.773784e+06,3.190587e+06,2.920785e+06,2.971258e+06,5.484214e+07,3.666129e+06,3.583081e+06,3.716801e+06,3.767771e+06,7.140854e+07,2.477318e+06,2.654789e+06,2.679026e+06,2.611952e+06,4.922980e+07
7,1,2.680801e+06,2.808346e+06,2.738861e+06,2.897882e+06,5.483004e+07,3.665946e+06,3.622910e+06,3.708004e+06,3.720475e+06,7.135439e+07,2.412385e+06,2.381265e+06,2.461805e+06,2.604572e+06,4.907828e+07
8,10,2.684446e+06,2.849210e+06,2.850490e+06,2.822901e+06,5.487951e+07,3.659519e+06,3.623287e+06,3.715671e+06,3.743246e+06,7.136141e+07,2.423445e+06,2.425430e+06,2.590360e+06,2.452005e+06,4.889627e+07
9,3,2.785067e+06,2.680150e+06,2.682375e+06,2.909664e+06,5.449788e+07,3.659896e+06,3.614838e+06,3.714038e+06,3.767241e+06,7.125338e+07,2.491374e+06,2.308342e+06,2.424244e+06,2.470129e+06,4.871997e+07


In [10]:
sum(df['base_gmv_1'])

54932159.35950501

In [11]:
sum(df['base_cost_1'])

73355571.27937499

In [12]:
sum(df['base_cost'])

1427396273.3433099

In [13]:
def cal(df_in,df_out,metric):
    df1=df_in.sort_values(by=['bucket_id'])
    lift_rate_list=[]
    for i in range(0,20):
        df_temp=df1[df1.bucket_id!=i]
        test_gmv_1=df_temp['test_{}_1'.format(metric)].values
        test_gmv_2=df_temp['test_{}_2'.format(metric)].values
        base_gmv_1=df_temp['base_{}_1'.format(metric)].values
        base_gmv_2=df_temp['base_{}_2'.format(metric)].values
        base_gmv=df_temp['base_{}'.format(metric)].values
        lift=(sum(test_gmv_1)-sum(test_gmv_2))-((sum(base_gmv_1)-sum(base_gmv_2)))
        lift_rate=lift/sum(base_gmv)
        lift_rate_list.append(lift_rate)
        
    var=(20*sum(map(lambda x:x**2,lift_rate_list))-sum(lift_rate_list)**2)*19/20/20
    # print(var)
    test_gmv_1=df1['test_{}_1'.format(metric)].values
    test_gmv_2=df1['test_{}_2'.format(metric)].values
    base_gmv_1=df1['base_{}_1'.format(metric)].values
    base_gmv_2=df1['base_{}_2'.format(metric)].values
    base_gmv=df1['base_{}'.format(metric)].values
    lift=(sum(test_gmv_1)-sum(test_gmv_2))-((sum(base_gmv_1)-sum(base_gmv_2)))
    lift_rate=lift/sum(base_gmv)
    lower_conf=lift_rate-1.96*math.sqrt(var)
    upper_conf=lift_rate+1.96*math.sqrt(var)
    is_sign=1 if lower_conf>0 or upper_conf<0 else 0
    
    size=df_out.index.size
    df_out.loc[size] = [endDate,expGroupId,metric,lift,lift_rate,var,lower_conf,upper_conf,is_sign]

In [14]:
df_out=pd.DataFrame(columns=['partition_date','exp_group_id','metric','lift','lift_rate','variance','lower_conf','upper_conf','is_sign'], index=[])

In [15]:
cal(df,df_out,'gmv')
cal(df,df_out,'cost')
cal(df,df_out,'gmv_normal')

In [16]:
df_out

,partition_date,exp_group_id,metric,lift,lift_rate,variance,lower_conf,upper_conf,is_sign
0,20220825,55072,gmv,1.177015e+05,0.000107,8.825939e-07,-0.001734,0.001949,0
1,20220825,55072,cost,-1.222571e+06,-0.000857,1.015596e-08,-0.001054,-0.000659,1
2,20220825,55072,gmv_normal,-7.571953e+05,-0.000772,5.769765e-07,-0.002261,0.000717,0
